In [ ]:
import pandas as pd
import os
import seaborn as sns

In [ ]:
bp = pd.read_csv('/Users/hari/Data/Building_Permits_Issued.csv')

In [ ]:
bp.head()

In [ ]:
building = bp.loc[bp['Permit Type Description'].str.contains('Building')]

In [ ]:
new_building = building.loc[building['Permit Type Description'].str.contains('New')]

In [ ]:
demolition = bp.loc[bp['Permit Type Description'].str.contains('Demolition')]

In [ ]:
res_demolition = demolition.loc[demolition['Permit Subtype Description'].str.contains('Residential')]

In [ ]:
merged = pd.merge(new_building, res_demolition, how='outer')

In [ ]:
#bp.loc[bp['Permit Subtype'].isin(['CAZ01A001', 'CAZ01A002'])]

In [ ]:
merged_sf = merged.loc[merged['Permit Subtype Description'].str.contains('Single Family Residence')]

In [ ]:
merged_sf.head()

In [ ]:
new_commercial = ['CAA03R299', 'CAA03R301', 'CAA03R398', 'CAA03R399', 'CAA05R202']

In [ ]:
new_residential = ['CAA01R301', 'CAA02R302', 'CAA03R298', 'CAA03R299', 'CAA03R301', 'CAA03R398', 'CAA03R399', 'CAA04R301', 'CAA05R202', 'CAA08R301', 'CAA10R301', 'CAZ08A001', 'DPLX2CNDO']

In [ ]:
matched = pd.merge(bp.loc[bp['Permit Subtype'].isin(['CAZ01A001', 'CAZ01A002'])], new_building.loc[new_building['Permit Subtype'].isin(new_commercial + new_residential)], on = 'Address')


In [ ]:
#bp['Permit Type Description'].value_counts()

In [ ]:
#bp['Permit Subtype Description'].value_counts()

In [ ]:
commercialdemo = bp.loc[bp['Permit Subtype']== 'CAZ01A001']

In [ ]:
sf_2009 = pd.read_csv('/Users/hari/Data/2009 single family_final_with_sqft.txt')
sf_2013 = pd.read_csv('/Users/hari/Data/2013 single family_final_with_sqft.txt')
sf_2017 = pd.read_csv('/Users/hari/Data/2017 single family_final_with_sqft.txt')

In [ ]:
sf_2017.loc[sf_2017.AddressFullAddress== '605 SUMMIT OAKS CT']

#building = bp.loc[bp['Permit Type Description'].str.contains('Building')]

In [ ]:
print(sf_2009.shape)
print(sf_2009.columns)
print(sf_2009.duplicated().sum())
#sf_2009.loc[sf_2009.duplicated(),:]

In [ ]:
print(sf_2013.shape)
print(sf_2013.columns)
print(sf_2013.duplicated().sum())


In [ ]:
print(sf_2017.shape)
print(sf_2017.duplicated().sum())
sf_2017.sort_values(ascending =False, by ='2017 TOTAL APPR').describe()

In [ ]:
sf_2017.isnull().sum()

In [ ]:
sf1 = pd.merge(sf_2009, sf_2013, on= 'AddressFullAddress', how = 'outer',suffixes = ('_f','_s'))
sf1.columns

In [ ]:
sf1['APN']= sf1['APN_f'].where(sf1['APN_s'].isnull(), sf1['APN_s'])
sf1['DistrictCode']= sf1['DistrictCode_f'].where(sf1['DistrictCode_s'].isnull(), sf1['DistrictCode_s'])
sf1['Council District']= sf1['Council District_f'].where(sf1['Council District_s'].isnull(), sf1['Council District_s'])
sf1['AddressCity']= sf1['AddressCity_f'].where(sf1['AddressCity_s'].isnull(), sf1['AddressCity_s'])
sf1['AddressPostalCode']= sf1['AddressPostalCode_f'].where(sf1['AddressPostalCode_s'].isnull(), sf1['AddressPostalCode_s'])
sf1['FinishedArea']= sf1['FinishedArea_f'].where(sf1['FinishedArea_s'].isnull(), sf1['FinishedArea_s'])
sf1.columns

In [ ]:
sf1.drop(['APN_f','DistrictCode_f','Council District_f','AddressCity_f',
          'AddressPostalCode_f', 'FinishedArea_f','DistrictCode_s','Council District_s','AddressCity_s','APN_s','AddressPostalCode_s','FinishedArea_s'], axis = 1,inplace = True)


In [ ]:
sf1.columns

In [ ]:
final_sf = pd.merge(sf1, sf_2017,on= 'AddressFullAddress', how = 'outer', suffixes = ('_m','_n'),indicator = True)
final_sf.columns

In [ ]:
final_sf['APN']= final_sf['APN_m'].where(final_sf['APN_n'].isnull(), final_sf['APN_n'])
final_sf['DistrictCode']= final_sf['DistrictCode_m'].where(final_sf['DistrictCode_n'].isnull(), final_sf['DistrictCode_n'])
final_sf['Council District']= final_sf['Council District_m'].where(final_sf['Council District_n'].isnull(), final_sf['Council District_n'])
final_sf['AddressCity']= final_sf['AddressCity_m'].where(final_sf['AddressCity_n'].isnull(), final_sf['AddressCity_n'])
final_sf['AddressPostalCode']= final_sf['AddressPostalCode_m'].where(final_sf['AddressPostalCode_n'].isnull(), final_sf['AddressPostalCode_n'])
final_sf['FinishedArea']= final_sf['FinishedArea_m'].where(final_sf['FinishedArea_n'].isnull(), final_sf['FinishedArea_n'])

In [ ]:
final_sf.columns

In [ ]:
final_sf.drop(['APN_m','DistrictCode_m','Council District_m','AddressCity_m',
          'AddressPostalCode_m', 'FinishedArea_m','DistrictCode_n','Council District_n','AddressCity_n','APN_n','AddressPostalCode_n','FinishedArea_n'], axis = 1, inplace = True)
final_sf.columns


In [ ]:
final_sf.shape

In [ ]:
final_sf.duplicated().sum()

In [ ]:
final_sf.loc[final_sf.duplicated(),:].shape

In [ ]:
final_sf.drop_duplicates(keep ='first').shape

In [ ]:
# final_sf.isnull().sum(axis = 0)
# count_nan = len(final_sf) - final_sf.count()
final_sf.columns

In [ ]:
final_sf['Finishedsqft']= final_sf['FinishedArea_f'].where(final_sf['FinishedArea_s'].isnull(), final_sf['FinishedArea_s'])
final_sf.head()


In [ ]:
# final_sf.drop(final_sf.loc['APN_y','DistrictCode_y', 'Council District_y',
#                'AddressCity_y','AddressPostalCode_y', 'FinishedArea_y'],axis=1, inplace = True)

In [ ]:
tax_2016 = pd.read_csv('/Users/hari/Data/2016tax.csv')

In [ ]:
tax_2016.loc[tax_2016.STATE=='TN']